In [1]:
import pandas as pd
df = pd.read_csv('../6.Data/kaggle_b2_fraud_train_v3.csv')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

X = df.drop(columns=['target_is_fraud'])
y = df['target_is_fraud']
knn = KNeighborsRegressor()
scores = cross_val_score(knn, X, y, cv=5, scoring='r2')
base_knn_score = scores.mean()
base_knn_score


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\yannc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\yannc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\yannc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_regression.py", line 227, in fit
    return self._fit(X, y)
           ^^^^^^^^^^^^^^^
  File "c:\Users\yannc\AppData\Local\Prog